In [1]:
import pandas as pd
import seaborn as sns


Replace the string inside the parentheses below with the path of your data. The data should be generated from "Cloud_Vision_API"

In [2]:
df_ori=pd.read_csv('/Users/gary.zhou/Documents/cloud_vision/Cloud_Vision_API.csv')

# The code below create a "index" column
def create_index(df):
    ind=0
    obj=df_ori['video_name/image_name'][0]
    index_list=[]
    for row in df_ori['video_name/image_name']:
        if row ==obj:
            index_list.append(ind)
        else:
            ind+=1
            obj=row
            index_list.append(ind)
    df_ori['index']=pd.Series(index_list)
    return df
df_ori=create_index(df_ori)


In [3]:
def concur(obj1,obj2,df):
    num=0
    ind=0
    con=0
    for row in df.itertuples():
        if row.index != ind:
            ind+=1
            con=0
        if (row.label==obj1)|(row.label==obj2):
            con+=1
            if con==2:
                num+=1
                con=0
    return num

def matrix_generate(matrix_object,df):
    df_final=pd.DataFrame()
    for obj in matrix_object:
        column=[]
        for obj_occur in matrix_object:
            num=concur(obj,obj_occur,df)
            column.append(num)
        df_final[obj]=column
    return df_final




In [4]:
def condition(df,frequncy,prob=0):
    df_1=df[df.probability>prob].groupby('label').count().sort_values('filename',ascending=False)
    matrix_obj=df_1[df_1['filename']>=frequncy].index.tolist()
    df=df[df.probability>prob]
    df.drop_duplicates(subset=['video_name/image_name','label'],inplace=True)
    return (matrix_obj,df)


In [5]:
def percentage_matrix(df,count):
    c=0
    for column in df:
        df[column]=df[column]/count[c]
        c+=1
    return df

Input the threshold inside the parentheses in the cell below:
(dataframe(don't change),frequncy,probability(default is 0 if you don't put any numbers here))

In [6]:

matrix_object,df_o=condition(df_ori,2,0.8)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
csv_print=matrix_generate(matrix_object,df_o)
df_matrix=csv_print
total=df_ori.groupby('label').count().sort_values('filename',ascending=False)['filename'].tolist()[0:len(matrix_object)]
final=percentage_matrix(df_matrix,total)
s=final.set_index([matrix_object])
per=s.style.format("{:,.2%}")
cm = sns.light_palette("green", as_cmap=True)
matrix = per.background_gradient(cmap=cm)


Matrix sample and interpretation 

In [9]:
matrix
'''Interpretation: Each number x in the matrix means that among all the videos/images which contain the label shown 
in the column name there are x percent of them contain the label shown in the row name. 
For example, the25% in the first row and second column indicates that among all the videos and images which have label
"public sapce"(column name),25% of them also have label "crowd"(row name).
'''

,crowd,public space,day,people,city,fun,community,metropolitan area,building,blue,urban area,sky
crowd,0.00%,25.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,50.00%,50.00%,50.00%
public space,25.00%,0.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,50.00%,50.00%,50.00%
day,25.00%,25.00%,0.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,50.00%,50.00%,50.00%
people,25.00%,25.00%,33.33%,0.00%,33.33%,33.33%,33.33%,33.33%,33.33%,50.00%,50.00%,50.00%
city,25.00%,25.00%,33.33%,33.33%,0.00%,33.33%,33.33%,66.67%,33.33%,50.00%,100.00%,50.00%
fun,25.00%,25.00%,33.33%,33.33%,33.33%,0.00%,33.33%,33.33%,33.33%,50.00%,50.00%,50.00%
community,25.00%,25.00%,33.33%,33.33%,33.33%,33.33%,0.00%,33.33%,33.33%,50.00%,50.00%,50.00%
metropolitan area,25.00%,25.00%,33.33%,33.33%,66.67%,33.33%,33.33%,0.00%,33.33%,50.00%,100.00%,50.00%
building,25.00%,25.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,0.00%,50.00%,50.00%,50.00%
blue,25.00%,25.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,0.00%,50.00%,50.00%


Run the code below to generate the matrix in .xlsx format

In [ ]:
matrix.to_excel('Label_Matrix.xlsx')
